$\newcommand{\giraffe}{\texttt{GiRaFFE}}$
# $\giraffe$: Solving GRFFE equations at a higher Finite Differencing order

### NRPy+ Source Code for this module: [Tutorial-GiRaFFE_Higher_Order.py](../edit/Tutorial-GiRaFFE_Higher_Order.py), which is fully documented in the [previous NRPy+ tutorial module](Tutorial-GiRaFFE_Higher_Order.ipynb) on using NRPy+ to construct Maxwell's equations and initial data as SymPy expressions.

This module focuses on using the equations developed in the [last tutorial](Tutorial-GiRaFFE_Higher_Order.ipynb) to build an Einstein Toolkit (ETK) thorn to solve the GRFFE equations in Cartesian coordinates. This tutorial will focus on implementing the time evolution aspects; others can be contructed to set up specific initial data.

When interfaced properly with the ETK, this module will propagate the initial data for $\tilde{S}_i$, $A_i$, and $\sqrt{\gamma} \Phi$, defined in the last tutorial, forward in time by integrating the equations for $\partial_t \tilde{S}_i$, $\partial_t A_i$ and $\partial_t [\sqrt{\gamma} \Phi]$ subject to spatial boundary conditions. The time evolution itself is handled by the $\text{MoL}$ (Method of Lines) thorn in the $\text{CactusNumerical}$ arrangement, and the boundary conditions by the $\text{Boundary}$ thorn in the $\text{CactusBase}$ arrangement. 

Similar to the other ETK modules we have built, we will construct the WaveToyNRPy module in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the evolution equations into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.


In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
par.set_parval_from_str("outputC::outCverbose",False) 

# Step 1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)

# Step 1c: Call the MaxwellCartesian_Evol() function from within the
#          Maxwell/MaxwellCartesian_Evol.py module.
import GiRaFFE_Higher_Order as gho
gho.GiRaFFE_Higher_Order()

# Step 2: Create the C code output kernel.
Prereqs_to_print = [\
                   lhrh(lhs=gri.gfaccess("out_gfs","uU0"),rhs=gho.uU[0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","uU1"),rhs=gho.uU[1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","uU2"),rhs=gho.uU[2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","alphau0"),rhs=gho.alphau0),\
                   lhrh(lhs=gri.gfaccess("out_gfs","alpsqrtgam"),rhs=gho.alpsqrtgam),\
                   lhrh(lhs=gri.gfaccess("out_gfs","AevolParen"),rhs=gho.AevolParen),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU0"),rhs=gho.PevolParenU[0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU1"),rhs=gho.PevolParenU[1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU2"),rhs=gho.PevolParenU[2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU00"),rhs=gho.gammaUU[0][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU01"),rhs=gho.gammaUU[0][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU02"),rhs=gho.gammaUU[0][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU11"),rhs=gho.gammaUU[1][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU12"),rhs=gho.gammaUU[1][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU22"),rhs=gho.gammaUU[2][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammadet"),rhs=gho.gammadet),\
                   ]

Conservs_to_print = [\
                    lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD0"),rhs=gho.Stilde_rhsD[0]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD1"),rhs=gho.Stilde_rhsD[1]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD2"),rhs=gho.Stilde_rhsD[2]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD0"),rhs=gho.A_rhsD[0]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD1"),rhs=gho.A_rhsD[1]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD2"),rhs=gho.A_rhsD[2]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","psi6Phi_rhs"),rhs=gho.psi6Phi_rhs),\
                    ]

Prereqs_CKernel = fin.FD_outputC("returnstring",Prereqs_to_print)
Conservs_CKernel = fin.FD_outputC("returnstring",Conservs_to_print)

Prereqs_looped = loop.loop(["i2","i1","i0"],["0","0","0"],\
                           ["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                           ["1","1","1"],["#pragma omp parallel for","",""],"",\
                           Prereqs_CKernel.replace("time","cctk_time"))

Conservs_looped = loop.loop(["i2","i1","i0"],["cctk_nghostzones[2]","cctk_nghostzones[1]","cctk_nghostzones[0]"],\
                            ["cctk_lsh[2]-cctk_nghostzones[2]","cctk_lsh[1]-cctk_nghostzones[1]",\
                             "cctk_lsh[0]-cctk_nghostzones[0]"],\
                            ["1","1","1"],["#pragma omp parallel for","",""],"",\
                            Conservs_CKernel.replace("time","cctk_time"))

# Step 3: Create directories for the thorn if they don't exist.
!mkdir GiRaFFE_HO     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir GiRaFFE_HO/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 4: Write the C code kernel to file.
with open("GiRaFFE_HO/src/Prereqs.h", "w") as file:
    file.write(str(Prereqs_looped))

with open("GiRaFFE_HO/src/Conservs.h", "w") as file:
    file.write(str(Conservs_looped))


We will also need a routine to compute new Valencia 3-velocities at each timestep using a conservative-to-primitive solver. Since we need $v^i_{(n)}$ everywhere, this will require us to compute $B^i$ everywhere. However, $B^i = \epsilon^{ijk} \partial_j A_k$ requires derivatives of $A_i$, so getting $B^i$ will require some finesse. A chief aspect of this will require using lower-order finite differencing in the ghost zones. To that end, we will create .h files for each finite differencing order $\leq 10$, as well as upwinded- and downwinded-derivatives at 2nd order. These will let us compute the derivative at the outermost gridpoints.

In [2]:
#AD = ixp.register_gridfunctions_for_single_rank1("AUX","AD",DIM=3)
# Step 2: Import the four metric

# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gho.gammadet)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gho.gammadet)

AD_dD = ixp.declarerank2("AD_dD","nosym")
BU = ixp.register_gridfunctions_for_single_rank1("AUX","BU")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 10)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_10.h",[\
                                                         lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                         lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                         lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 8)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_8.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 6)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_6.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_4.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

AD_ddnD = ixp.declarerank2("AD_ddnD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x0D.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])
AD_dupD = ixp.declarerank2("AD_dupD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x0U.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x1D.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x1U.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x2D.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x2U.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])



Wrote to file "GiRaFFE_HO/src/B_from_A_10.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_8.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_6.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_4.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x0D.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x0U.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x1D.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x1U.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x2D.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x2U.h"


### Step 2: Interfacing with the Einstein Toolkit

#### Step 2a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels.

Now that we have generated the C code kernel *GiRaFFE_RHSs.h* and the parameters file *NRPy_params.h*, we will need to write C code to make use of these files. To do this, we can simply follow the example within the [IDScalarWaveNRPy tutorial module](Tutorial-ETK_thorn-IDScalarWaveNRPy.ipynb). Functions defined by these files will be called by the Einstein Toolkit scheduler (specified in schedule.ccl below).

Also, here we will write the logic that determines which files are called where in order to calculate $B^i$. 
1. Take the primary finite differencing order $N$ from the param.ccl file. Fill in the interior points with the corresponding FD order. 
1. Then, at each face, at $0+\text{cctk_nghostzones[face]}-1$ and $\text{cctk_lsh[face]}-\text{cctk_nghostzones[face]}+1$, calculate B at order $N-2$
1. Continue moving outwards: at the points $0+\text{cctk_nghostzones[face]}-p$ and $\text{cctk_lsh[face]}-\text{cctk_nghostzones[face]}+p$, calculate B at order $N-2p$.
1. When $\text{cctk_nghostzones[face]}-p = 0$, use the upwinding derivatives 

In [3]:
%%writefile GiRaFFE_HO/src/GiRaFFE.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

const int MAXFACE = -1;
const int NUL     = +0;
const int MINFACE = +1;


// Declare boundary condition FACE_UPDATE function,
// which fills in the ghost zones with successively
// lower order finite differencing
void AtoB(const int ORDER, 
          const CCTK_REAL *AD0GF,const CCTK_REAL *AD1GF,const CCTK_REAL *AD2GF,
          const CCTK_REAL *BU0GF,const CCTK_REAL *BU1GF,const CCTK_REAL *BU2GF,
          const int i0min, const int i0max, 
          const int i1min, const int i1max, 
          const int i2min, const int i2max, 
          const int FACEX0, const int FACEX1, const int FACEX2) {
  
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
    
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  const CCTK_REAL *gammaDD00GF = gxx;
  const CCTK_REAL *gammaDD01GF = gxy;
  const CCTK_REAL *gammaDD02GF = gxz;
  const CCTK_REAL *gammaDD11GF = gyy;
  const CCTK_REAL *gammaDD12GF = gyz;
  const CCTK_REAL *gammaDD22GF = gzz;

  if(ORDER==8) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_8.h"
    }
  } else if(ORDER==6) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_6.h"
    }
  } else if(ORDER==4) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_4.h"
    }
  } else if(ORDER==2) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_2.h"
    } 
  } else if(ORDER==0) {
    if(FACEX0==MAXFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0D.h"
        }
    } else if(FACEX0==MINFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0U.h"
        }
    } else if(FACEX1==MAXFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1D.h"
        }
    } else if(FACEX1==MINFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1U.h"
        }
    } else if(FACEX2==MAXFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2D.h"
        }
    } else if(FACEX2==MINFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2U.h"
        }
    } else {
        printf("ERROR. FACEX parameters not set properly.\n");
        exit(1);
    }
  } else {
    printf("ERROR. ORDER = %d not supported!\n",ORDER);
    exit(1);
  }
}

void driver_A_to_B(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const int *NG = cctk_nghostzones;
  const int *Nx = cctk_lsh;
  CCTK_INT ORDER = NG[0]*2;
  for(int ii=0;ii<cctk_lsh[2]*cctk_lsh[1]*cctk_lsh[0];ii++) {
      BU0[ii] = 1.0 / 0.0;
      BU1[ii] = 1.0 / 0.0;
      BU2[ii] = 1.0 / 0.0;
  }

  AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, NG[0],Nx[0]-NG[0],NG[1],Nx[1]-NG[1],NG[2],Nx[2]-NG[2], NUL,NUL,NUL);
  int imin[3] = { NG[0], NG[1], NG[2] };
  int imax[3] = { Nx[0]-NG[0], Nx[1]-NG[1], Nx[2]-NG[2] };
  while(ORDER>0) {
      // After updating each face, adjust imin[] and imax[] 
      //   to reflect the newly-updated face extents.
      ORDER -= 2;
      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL); 
      if(ORDER!=0) imin[0]--;
      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imax[0],imax[0]+1, imin[1],imax[1], imin[2],imax[2], MAXFACE,NUL,NUL); 
      if(ORDER!=0) imax[0]++;

      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imin[1]-1,imin[1], imin[2],imax[2], NUL,MINFACE,NUL); 
      if(ORDER!=0) imin[1]--;
      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imax[1],imax[1]+1, imin[2],imax[2], NUL,MAXFACE,NUL); 
      if(ORDER!=0) imax[1]++;

      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imin[1],imax[1], imin[2]-1,imin[2], NUL,NUL,MINFACE); 
      if(ORDER!=0) imin[2]--;
      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imin[1],imax[1], imax[2],imax[2]+1, NUL,NUL,MAXFACE); 
      if(ORDER!=0) imax[2]++;
    }
}

void GiRaFFE_HO_calc_prereqs(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                             const CCTK_REAL invdx0,const CCTK_REAL invdx1,const CCTK_REAL invdx2,
                             const CCTK_REAL *alphaGF, const CCTK_REAL *betaU0GF, const CCTK_REAL *betaU1GF, const CCTK_REAL *betaU2GF,
                             const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                             const CCTK_REAL *ValenciavU0GF, const CCTK_REAL *ValenciavU1GF, const CCTK_REAL *ValenciavU2GF, const CCTK_REAL *u0,
                             const CCTK_REAL *AD0GF,const CCTK_REAL *AD1GF,const CCTK_REAL *AD2GF,const CCTK_REAL *psi6PhiGF,
                             CCTK_REAL *gammaUU00GF,CCTK_REAL *gammaUU01GF,CCTK_REAL *gammaUU02GF,CCTK_REAL *gammaUU11GF,CCTK_REAL *gammaUU12GF,CCTK_REAL *gammaUU22GF,CCTK_REAL *gammadetGF,
                             CCTK_REAL *uU0GF, CCTK_REAL *uU1GF, CCTK_REAL *uU2GF,
                             CCTK_REAL *alphau0GF, CCTK_REAL *alpsqrtgamGF, CCTK_REAL *AevolParenGF,
                             CCTK_REAL *PevolParenU0GF,CCTK_REAL *PevolParenU1GF,CCTK_REAL *PevolParenU2GF) {

  DECLARE_CCTK_PARAMETERS;

#include "Prereqs.h" 

}

void GiRaFFE_HO_calc_rhs(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                         const CCTK_REAL invdx0,const CCTK_REAL invdx1,const CCTK_REAL invdx2,
                         const CCTK_REAL *alphaGF, const CCTK_REAL *betaU0GF, const CCTK_REAL *betaU1GF, const CCTK_REAL *betaU2GF,
                         const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                         const CCTK_REAL *gammaUU00GF,const CCTK_REAL *gammaUU01GF,const CCTK_REAL *gammaUU02GF,const CCTK_REAL *gammaUU11GF,const CCTK_REAL *gammaUU12GF,const CCTK_REAL *gammaUU22GF,const CCTK_REAL *gammadetGF,
                         const CCTK_REAL *ValenciavU0GF, const CCTK_REAL *ValenciavU1GF, const CCTK_REAL *ValenciavU2GF, const CCTK_REAL *u0,
                         const CCTK_REAL *uU0GF, const CCTK_REAL *uU1GF, const CCTK_REAL *uU2GF, 
                         const CCTK_REAL *AD0GF,const CCTK_REAL *AD1GF,const CCTK_REAL *AD2GF,const CCTK_REAL *psi6PhiGF,
                         const CCTK_REAL *alphau0GF, const CCTK_REAL *alpsqrtgamGF, const CCTK_REAL *AevolParenGF,
                         const CCTK_REAL *PevolParenU0GF,const CCTK_REAL *PevolParenU1GF,const CCTK_REAL *PevolParenU2GF,
                         CCTK_REAL *Stilde_rhsD0GF, CCTK_REAL *Stilde_rhsD1GF, CCTK_REAL *Stilde_rhsD2GF,
                         CCTK_REAL *A_rhsD0GF, CCTK_REAL *A_rhsD1GF, CCTK_REAL *A_rhsD2GF, CCTK_REAL *psi6Phi_rhsGF) {
  DECLARE_CCTK_PARAMETERS;

#include "Conservs.h"

}

void GiRaFFE_HO_set_prereqs(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  
  GiRaFFE_HO_calc_prereqs(cctkGH,cctk_lsh,cctk_nghostzones,
                          invdx0, invdx1, invdx2,
                          alp, betax, betay, betaz,
                          gxx, gxy, gxz, gyy, gyz, gzz,
                          gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet,
                          ValenciavU0, ValenciavU1, ValenciavU2, u0,
                          AD0, AD1, AD2, psi6Phi,
                          uU0, uU1, uU2,
                          alphau0, alpsqrtgam, AevolParen,
                          PevolParenU0, PevolParenU1, PevolParenU2);
}

void GiRaFFE_HO_set_rhs(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  
  GiRaFFE_HO_calc_rhs(cctkGH,cctk_lsh,cctk_nghostzones,
                      invdx0, invdx1, invdx2,
                      alp, betax, betay, betaz,
                      gxx, gxy, gxz, gyy, gyz, gzz,
                      gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet,
                      ValenciavU0, ValenciavU1, ValenciavU2, u0,
                      uU0, uU1, uU2,
                      AD0, AD1, AD2, psi6Phi,
                      alphau0, alpsqrtgam, AevolParen,
                      PevolParenU0, PevolParenU1, PevolParenU2,
                      Stilde_rhsD0, Stilde_rhsD1, Stilde_rhsD2,
                      A_rhsD0, A_rhsD1, A_rhsD2, psi6Phi_rhs);
}

/* Boundary Condition code adapted from WaveToyC thorn in ETK, implementing built-in
 * ETK BC functionality
 */
void GiRaFFE_HO_SelectBCs(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  const char *bctype;


  bctype = NULL;
  if (CCTK_EQUALS(bound,"flat") || CCTK_EQUALS(bound,"static") ||
      CCTK_EQUALS(bound,"radiation") || CCTK_EQUALS(bound,"robin") ||
      CCTK_EQUALS(bound,"none"))
  {
    bctype = bound;
  }
  else if (CCTK_EQUALS(bound,"zero"))
  {
    bctype = "scalar";
  }

  /* Uses all default arguments, so invalid table handle -1 can be passed */
  /*TODO: Add correct gridfunctions!*/
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::Stilde_rhsD0", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::Stilde_rhsD1", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::Stilde_rhsD2", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::A_rhsD0", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::A_rhsD1", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::A_rhsD2", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::psi6Phi_rhs", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
}

void GiRaFFE_HO_InitSymBound(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
      
  int sym[3];

  sym[0] = 1;
  sym[1] = 1;
  sym[2] = 1;

  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::uU0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::uU1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::uU2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::alphau0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::alpsqrtgam");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AevolParen");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PevolParenU0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PevolParenU1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PevolParenU2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PsitildeD0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PsitildeD1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PsitildeD2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AD0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AD1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AD2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::psi6Phi");

  return;
}

void GiRaFFE_HO_RegisterVars(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  CCTK_INT ierr CCTK_ATTRIBUTE_UNUSED = 0;
  /* Register all the evolved grid functions with MoL */
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::PsitildeD0"),  CCTK_VarIndex("GiRaFFE_HO::Psitilde_rhsD0"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::PsitildeD1"),  CCTK_VarIndex("GiRaFFE_HO::Psitilde_rhsD1"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::PsitildeD2"),  CCTK_VarIndex("GiRaFFE_HO::Psitilde_rhsD2"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::AD0"),  CCTK_VarIndex("GiRaFFE_HO::A_rhsD0"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::AD1"),  CCTK_VarIndex("GiRaFFE_HO::A_rhsD1"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::AD2"),  CCTK_VarIndex("GiRaFFE_HO::A_rhsD2"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::psi6Phi"),  CCTK_VarIndex("GiRaFFE_HO::psi6Phi_rhs"));
  /* Register all the evolved Array functions with MoL */
  return;
}

Overwriting GiRaFFE_HO/src/GiRaFFE.c


#### Step 2b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. This file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist within those functions. Then, we tell the toolkit that we want the gridfunctions $A_i$, $\tilde{S}_i$, and $\sqrt{\gamma}\Phi$ to be visible to other thorns by using the keyword "public". 

In [4]:
%%writefile GiRaFFE_HO/interface.ccl
implements: GiRaFFE_HO

inherits: admbase Boundary grid

USES INCLUDE: loopcontrol.h
USES INCLUDE: Symmetry.h
USES INCLUDE: Boundary.h
    
CCTK_INT FUNCTION MoLRegisterEvolved(CCTK_INT IN EvolvedIndex, CCTK_INT IN RHSIndex)
USES FUNCTION MoLRegisterEvolved

CCTK_INT FUNCTION GetBoundarySpecification(CCTK_INT IN size, CCTK_INT OUT ARRAY nboundaryzones, CCTK_INT OUT ARRAY is_internal, CCTK_INT OUT ARRAY is_staggered, CCTK_INT OUT ARRAY shiftout)
USES FUNCTION GetBoundarySpecification

CCTK_INT FUNCTION SymmetryTableHandleForGrid(CCTK_POINTER_TO_CONST IN cctkGH)
USES FUNCTION SymmetryTableHandleForGrid

CCTK_INT FUNCTION Boundary_SelectGroupForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN group_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectGroupForBC

CCTK_INT FUNCTION Boundary_SelectVarForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN var_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectVarForBC

public:
cctk_real GiRaFFE_aux type = GF Timelevels=1 tags='tensortypealias="Scalar"'
{
  uU0,uU1,uU2,alphau0,alpsqrtgam,AevolParen,PevolParenU0,PevolParenU1,PevolParenU2,
  gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet
} "The evolved scalar fields"

public:
cctk_real GiRaFFE_Bs type = GF Timelevels=1 tags='tensortypealias="Scalar"'
{
  BU0,BU1,BU2
} "The B field"

public:
cctk_real GiRaFFE_Vs type = GF Timelevels=1 tags='tensortypealias="Scalar"'
{
  ValenciavU0,ValenciavU1,ValenciavU2
} "The Valencia 3-velocity"

public:
cctk_real GiRaFFE_rhs type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  Stilde_rhsD0,Stilde_rhsD1,Stilde_rhsD2,A_rhsD0,A_rhsD1,A_rhsD2,psi6Phi_rhs
} "The evolved scalar fields"

public:
cctk_real GiRaFFE_vars type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  StildeD0,StildeD1,StildeD2,AD0,AD1,AD2,psi6Phi
} "The evolved scalar fields"

public:
cctk_real GiRaFFE_u0 type = SCALAR Timelevels=1 tags='tensortypealias="Scalar"'
{
  u0
} "The zeroth component of the four velocity"


Overwriting GiRaFFE_HO/interface.ccl


2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). A number of parameters are defined, and more parameters can be easily added in later versions. We also set the number of timelevels we will store in memory.

In [5]:
%%writefile GiRaFFE_HO/param.ccl
shares: MethodOfLines

USES CCTK_INT MoL_Num_Evolved_Vars
USES CCTK_INT MoL_Num_ArrayEvolved_Vars

restricted:
CCTK_INT GiRaFFE_HO_MaxNumEvolvedVars "Number of evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_Evolved_Vars STEERABLE=RECOVER
{
  7:7 :: "Number of evolved variables used by this thorn"
} 7

restricted:
CCTK_INT GiRaFFE_HO_MaxNumArrayEvolvedVars "Number of Array evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_ArrayEvolved_Vars STEERABLE=RECOVER
{
  0:0 :: "Number of Array evolved variables used by this thorn"
} 0

restricted:
KEYWORD bound "Type of boundary condition to use"
{
  "flat"      :: "Flat (von Neumann, n grad phi = 0) boundary condition"
  "static"    :: "Static (Dirichlet, dphi/dt=0) boundary condition"
  "radiation" :: "Radiation boundary condition"
  "robin"     :: "Robin (phi(r) = C/r) boundary condition"
  "zero"      :: "Zero (Dirichlet, phi=0) boundary condition"
  "none"      :: "Apply no boundary condition"
} "static"

restricted:
CCTK_INT timelevels "Number of active timelevels" STEERABLE=RECOVER
{
  0:3 :: ""
} 3

restricted:
CCTK_REAL xi "The damping factor for the psi6Phi evolution equation"
{
 *:* :: "The damping factor for the psi6Phi evolution equation"
} 1.0


Overwriting GiRaFFE_HO/param.ccl


3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We first assign storage for both scalar gridfunctions, and then specify the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [6]:
%%writefile GiRaFFE_HO/schedule.ccl
STORAGE: GiRaFFE_rhs[timelevels]
STORAGE: GiRaFFE_vars[timelevels]
STORAGE: GiRaFFE_aux[timelevels]
STORAGE: GiRaFFE_Bs[timelevels]

schedule GiRaFFE_HO_InitSymBound at BASEGRID
{
  LANG: C
  OPTIONS: global
} "Schedule symmetries"

schedule driver_A_to_B as driver_A_to_B IN MoL_CalcRHS
{
  LANG: C
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  WRITES: GiRaFFE_HO::BU0(Everywhere)
  WRITES: GiRaFFE_HO::BU1(Everywhere)
  WRITES: GiRaFFE_HO::BU2(Everywhere)
} "Calculates the B-field from the vector potential"

schedule GiRaFFE_HO_set_prereqs as GiRaFFE_HO_set_prereqs IN MoL_CalcRHS after driver_A_to_B
{
  LANG: C
  READS: admbase::alp(Everywhere)
  READS: admbase::betax(Everywhere)
  READS: admbase::betay(Everywhere)
  READS: admbase::betaz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  READS: GiRaFFE_HO::psi6Phi(Everywhere)
  WRITES: GiRaFFE_HO::uU0(Everywhere)
  WRITES: GiRaFFE_HO::uU1(Everywhere)
  WRITES: GiRaFFE_HO::uU2(Everywhere)
  WRITES: GiRaFFE_HO::alphau0(Everywhere)
  WRITES: GiRaFFE_HO::alpsqrtgam(Everywhere)
  WRITES: GiRaFFE_HO::AevolParen(Everywhere)
  WRITES: GiRaFFE_HO::PevolParenU0(Everywhere)
  WRITES: GiRaFFE_HO::PevolParenU1(Everywhere)
  WRITES: GiRaFFE_HO::PevolParenU2(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU00(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU01(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU02(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU11(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU12(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU22(Everywhere)
  WRITES: GiRaFFE_HO::gammadet(Everywhere)
  SYNC: GiRaFFE_aux
} "Sets prerequisite quantities for the GiRaFFE right-hand sides"

schedule GiRaFFE_HO_set_rhs as GiRaFFE_HO_Evolution IN MoL_CalcRHS after GiRaFFE_HO_set_prereqs
{
  LANG: C
  READS: admbase::alp(Everywhere)
  READS: admbase::betax(Everywhere)
  READS: admbase::betay(Everywhere)
  READS: admbase::betaz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  READS: GiRaFFE_HO::uU0(Everywhere)
  READS: GiRaFFE_HO::uU1(Everywhere)
  READS: GiRaFFE_HO::uU2(Everywhere)
  READS: GiRaFFE_HO::alphau0(Everywhere)
  READS: GiRaFFE_HO::alpsqrtgam(Everywhere)
  READS: GiRaFFE_HO::AevolParen(Everywhere)
  READS: GiRaFFE_HO::PevolParenU0(Everywhere)
  READS: GiRaFFE_HO::PevolParenU1(Everywhere)
  READS: GiRaFFE_HO::PevolParenU2(Everywhere)
  READS: GiRaFFE_HO::gammaUU00(Everywhere)
  READS: GiRaFFE_HO::gammaUU01(Everywhere)
  READS: GiRaFFE_HO::gammaUU02(Everywhere)
  READS: GiRaFFE_HO::gammaUU11(Everywhere)
  READS: GiRaFFE_HO::gammaUU12(Everywhere)
  READS: GiRaFFE_HO::gammaUU22(Everywhere)
  READS: GiRaFFE_HO::gammadet(Everywhere)
  WRITES: GiRaFFE_HO::Stilde_rhsD0(Interior)
  WRITES: GiRaFFE_HO::Stilde_rhsD1(Interior)
  WRITES: GiRaFFE_HO::Stilde_rhsD2(Interior)
  WRITES: GiRaFFE_HO::A_rhsD0(Interior)
  WRITES: GiRaFFE_HO::A_rhsD1(Interior)
  WRITES: GiRaFFE_HO::A_rhsD2(Interior)
  WRITES: GiRaFFE_HO::psi6Phi_rhs(Interior)
} "Sets the GiRaFFE right-hand sides"

schedule GiRaFFE_HO_SelectBCs in MoL_PostStep
{
  LANG: C
  OPTIONS: level
  SYNC: GiRaFFE_vars
} "Boundaries of Maxwell's equations"

schedule GROUP ApplyBCs as GiRaFFE_HO_ApplyBCs in MoL_PostStep after GiRaFFE_HO_SelectBCs
{
} "Apply boundary conditions"


schedule GROUP ApplyBCs as GiRaFFE_HO_ApplyBCs at POSTRESTRICT
{
} "Apply boundary conditions"

schedule GiRaFFE_HO_RegisterVars in MoL_Register
{
  LANG: C
  OPTIONS: meta
} "Register Variables for MoL"


Overwriting GiRaFFE_HO/schedule.ccl


#### Step 2c: Add the C file to Einstein Toolkit compilation list.

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.

In [7]:
%%writefile GiRaFFE_HO/src/make.code.defn
SRCS = GiRaFFE.c

Overwriting GiRaFFE_HO/src/make.code.defn
